In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df = pd.read_csv("../data/preproccesed/Climate_Data_joined.csv", index_col=0)

/tmp/ipykernel_30178/1151001539.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/preproccesed/Climate_Data_joined.csv", index_col=0)


In [3]:
df.iloc[:, 12].unique()

array([144.966492  , 144.964122  , 144.960923  , 144.964635  ,
       144.965052  , 144.952065  , 144.951835  , 144.96705703,
       144.96570467, 144.95222195, 144.952222  , 144.97      ,
       144.956389  , 144.967222  , 144.9725    , 144.960556  ])

In [69]:
# Get unique combinations
col_names = ["latitude_sensor","longitude_sensor"] 
df_locations = df.groupby(col_names)[col_names].size().reset_index(name="count")
df_locations

,latitude_sensor,longitude_sensor,count
0,-37.822500,144.952222,240167
1,-37.822486,144.952065,31392
2,-37.822460,144.951835,272184
3,-37.822222,144.952222,190870
4,-37.817500,144.967222,168268
5,-37.817465,144.967057,30160
6,-37.816868,144.965705,250229
7,-37.813333,144.970000,165004
8,-37.812778,144.960556,134850
9,-37.811944,144.956389,165282


In [124]:
fig = px.scatter_mapbox(df_locations,
                     lat="latitude_sensor",
                     lon="longitude_sensor",
                     hover_data={'latitude_sensor': False,
                                 "longitude_sensor": False,
                                 "count": ":100.0f"},
                     color_discrete_sequence=["darkgreen"],
                     size="count",
                     size_max=20,
                     zoom=13)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

fig.show()

In [125]:
df_locations.to_csv("../data/preproccesed/locations.csv")